
## Import 

In [1]:
cd ..

/home/alex/Desktop/openclassroom/mentorat/__alexandre_gazagnes_mentorat/DS/P7/for-nora


In [2]:
import os, sys, pickle

from IPython.display import display

import requests

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
import seaborn as sns
sns.set()
import plotly.express as px

from sklearn.model_selection import * 
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.impute import * 
from sklearn.preprocessing import * 
from sklearn.ensemble import * 
from xgboost import XGBClassifier



# import shap

## Load data

In [3]:
url = "./data/cleaned/df.csv"

In [4]:
df = pd.read_csv (url)

In [5]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


## Prepare

In [6]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [7]:
X = df.drop(columns="Survived")
y = df.Survived

In [8]:
df_train, df_test  = train_test_split(df)

In [9]:
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
139,140,0,1,male,24.0,0,0,79.2000,C
162,163,0,3,male,26.0,0,0,7.7750,S
581,582,1,1,female,39.0,1,1,110.8833,C
83,84,0,1,male,28.0,0,0,47.1000,S
93,94,0,3,male,26.0,1,2,20.5750,S


In [10]:
df_train.shape

(668, 9)

In [11]:
df_test.shape

(223, 9)

In [12]:
y_train = df_train.Survived.values

In [13]:
# X_train_PassengerId = df_train.PassengerId.values

X_train = df_train.drop(columns=["Survived", ], inplace=False)
X_train.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
139,140,1,male,24.0,0,0,79.2000,C
162,163,3,male,26.0,0,0,7.7750,S
581,582,1,female,39.0,1,1,110.8833,C
83,84,1,male,28.0,0,0,47.1000,S
93,94,3,male,26.0,1,2,20.5750,S


In [14]:
y_train[:10]

array([0, 0, 1, 0, 0, 0, 0, 0, 1, 1])

In [15]:
len(y_train)

668

In [16]:
len(X_train)

668

In [17]:
y_test = df_test.Survived.values


In [18]:
# X_test_PassengerId = df_test.PassengerId.values
X_test = df_test.drop(columns=["Survived" ], inplace=False)
X_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
183,184,2,male,1.0,2,1,39.0000,S
193,194,2,male,3.0,1,1,26.0000,S
363,364,3,male,35.0,0,0,7.0500,S
607,608,1,male,27.0,0,0,30.5000,S
269,270,1,female,35.0,0,0,135.6333,S


In [19]:
X_train.select_dtypes(include=np.number).columns

Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [20]:
X_train.select_dtypes(exclude=np.number).columns

Index(['Sex', 'Embarked'], dtype='object')

## Model

In [21]:
preprocessor = ColumnTransformer(transformers=[
    ("num", "passthrough",['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']),
    ("oneHot", OneHotEncoder( handle_unknown="ignore"), ['Sex', 'Embarked'])
    ], remainder="drop")

In [22]:
X_train.head(10)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
139,140,1,male,24.0,0,0,79.2000,C
162,163,3,male,26.0,0,0,7.7750,S
581,582,1,female,39.0,1,1,110.8833,C
83,84,1,male,28.0,0,0,47.1000,S
93,94,3,male,26.0,1,2,20.5750,S
191,192,2,male,19.0,0,0,13.0000,S
282,283,3,male,16.0,0,0,9.5000,S
439,440,2,male,31.0,0,0,10.5000,S
533,534,3,female,NaN,0,2,22.3583,C
32,33,3,female,NaN,0,0,7.7500,Q


In [23]:
tmp = preprocessor.fit_transform(X_train)
pd.DataFrame(tmp).head(10)

,0,1,2,3,4,5,6,7,8,9,10
0,1.0,24.0,0.0,0.0,79.2000,0.0,1.0,1.0,0.0,0.0,0.0
1,3.0,26.0,0.0,0.0,7.7750,0.0,1.0,0.0,0.0,1.0,0.0
2,1.0,39.0,1.0,1.0,110.8833,1.0,0.0,1.0,0.0,0.0,0.0
3,1.0,28.0,0.0,0.0,47.1000,0.0,1.0,0.0,0.0,1.0,0.0
4,3.0,26.0,1.0,2.0,20.5750,0.0,1.0,0.0,0.0,1.0,0.0
5,2.0,19.0,0.0,0.0,13.0000,0.0,1.0,0.0,0.0,1.0,0.0
6,3.0,16.0,0.0,0.0,9.5000,0.0,1.0,0.0,0.0,1.0,0.0
7,2.0,31.0,0.0,0.0,10.5000,0.0,1.0,0.0,0.0,1.0,0.0
8,3.0,NaN,0.0,2.0,22.3583,1.0,0.0,1.0,0.0,0.0,0.0
9,3.0,NaN,0.0,0.0,7.7500,1.0,0.0,0.0,1.0,0.0,0.0


In [24]:
imputer = KNNImputer()

# X_train = pd.DataFrame(imputer.transform(X_train), columns=X_train.columns)
# X_test = pd.DataFrame(imputer.transform(X_test), columns=X_train.columns)
# X_train.info()

In [25]:
pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("imputer",imputer),
    ("estimator", RandomForestClassifier())
])
pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('oneHot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Sex', 'Embarked'])])),
                ('imputer', KNNImputer()),
                ('estimator', RandomForestClassifier())])

In [26]:
grid = GridSearchCV(pipe, {}, cv=10, return_train_score=True, n_jobs=-1)

## Train

In [27]:
grid.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         'passthrough',
                                                                         ['Pclass',
                                                                          'Age',
                                                                          'SibSp',
                                                                          'Parch',
                                                                          'Fare']),
                                                                        ('oneHot',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Sex',
                                                                          'Embarked'])])),
                                       ('imputer', KNNImputer()),
                                       ('estimator',
                                        RandomForestClassifier())]),
             n_jobs=-1, param_grid={}, return_train_score=True)

In [28]:
res = pd.DataFrame(grid.cv_results_)
cols = [i for i in res.columns if ("split" not in i) ]
res = res.loc[:, cols]
res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
0,0.3182,0.062854,0.024128,0.011885,{},0.799389,0.036731,1,0.98636,0.001792


## Test

In [29]:
grid.score(X_train, y_train)

0.9850299401197605

In [30]:
model = grid.best_estimator_
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('oneHot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Sex', 'Embarked'])])),
                ('imputer', KNNImputer()),
                ('estimator', RandomForestClassifier())])

In [31]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('oneHot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Sex', 'Embarked'])])),
                ('imputer', KNNImputer()),
                ('estimator', RandomForestClassifier())])

In [32]:
model.score(X_test, y_test)

0.7937219730941704

In [33]:
pre = grid.best_estimator_['preprocessor']
pre


ColumnTransformer(transformers=[('num', 'passthrough',
                                 ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']),
                                ('oneHot',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['Sex', 'Embarked'])])

In [34]:
imp = pre = grid.best_estimator_['imputer']
imp

KNNImputer()

In [35]:
est = grid.best_estimator_['estimator']
est

RandomForestClassifier()

# export

In [36]:
with open("./models/model.pk", "wb") as f : 
    pk = pickle.dumps(model)
    f.write(pk)

In [37]:
with open("./models/preprocessor.pk", "wb") as f : 
    pk = pickle.dumps(pre)
    f.write(pk)

In [38]:
with open("./models/imputer.pk", "wb") as f : 
    pk = pickle.dumps(imp)
    f.write(pk)

In [39]:
with open("./models/estimator.pk", "wb") as f : 
    pk = pickle.dumps(est)
    f.write(pk)

In [40]:
display(len(X_train))
# display(len(X_train_PassengerId))
display(len(y_train))

668

668

In [41]:
len(y_train)

668

In [42]:
X_train["Survived"] = y_train
# X_train["PassengerId"] = X_train_PassengerId
X_train.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
139,140,1,male,24.0,0,0,79.2000,C,0
162,163,3,male,26.0,0,0,7.7750,S,0
581,582,1,female,39.0,1,1,110.8833,C,1
83,84,1,male,28.0,0,0,47.1000,S,0
93,94,3,male,26.0,1,2,20.5750,S,0


In [43]:
X_train.isna().sum()

PassengerId      0
Pclass           0
Sex              0
Age            142
SibSp            0
Parch            0
Fare             0
Embarked         2
Survived         0
dtype: int64

In [44]:
X_train.to_csv("./data/cleaned/df_train.csv", index=False)

In [45]:
X_test["Survived"] = y_test
# X_test["PassengerId"] = X_test_PassengerId
X_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
183,184,2,male,1.0,2,1,39.0000,S,1
193,194,2,male,3.0,1,1,26.0000,S,1
363,364,3,male,35.0,0,0,7.0500,S,0
607,608,1,male,27.0,0,0,30.5000,S,1
269,270,1,female,35.0,0,0,135.6333,S,1


In [46]:
X_test.isna().sum()

PassengerId     0
Pclass          0
Sex             0
Age            35
SibSp           0
Parch           0
Fare            0
Embarked        0
Survived        0
dtype: int64

In [47]:
X_test.to_csv("./data/cleaned/df_test.csv", index=False)